In [28]:
import numpy as np
from scipy.stats import binom
import math

data = np.array(([14, 33, 19, 10, 0, 17, 24, 17, 1, 36, 5, 6, 5, 13, 4, 35, 5, 5, 74, 34],
                 [41, 43, 23, 23, 1, 23, 36, 37, 2, 131, 5, 29, 13, 47, 10, 58, 15, 14, 100, 113]))    

In [29]:
import numpy as np
from scipy.stats import binom
import math

data = np.array(([14, 33, 19, 10, 0, 17, 24, 17, 1, 36, 5, 6, 5, 13, 4, 35, 5, 5, 74, 34],
                 [41, 43, 23, 23, 1, 23, 36, 37, 2, 131, 5, 29, 13, 47, 10, 58, 15, 14, 100, 113]))    
def EM(inits, data): 
    #E-step: calculate the expected probability for the latent variable assigning mechanism for each datapoint
    p_As, p_Bs = [], []
    for _ in range(data.shape[1]):
        p_A = binom.pmf(data[0,_],data[1,_],inits[0])/(binom.pmf(data[0,_],data[1,_],inits[0])
                                                      +binom.pmf(data[0,_],data[1,_],inits[1]))
        p_B = 1 - p_A
        p_As.append(p_A)
        p_Bs.append(p_B)
    #M-step: MLE for inits under the calculated expected probability of the latent variable
    p_As, p_Bs = np.array(p_As), np.array(p_Bs)
    updates = []
    updates.append(np.sum(p_As*data[0,:])/np.sum(p_As*data[1,:]))
    updates.append(np.sum(p_Bs*data[0,:])/np.sum(p_Bs*data[1,:]))
    return np.array(updates)

count = 0
inits = np.array([0.7 ,0.3])
check_A, check_B = 10, 10
while check_A > 0.00001 or check_B > 0.00001:
    updates = EM(inits,data)
    check_A, check_B = math.fabs(updates[0]-inits[0]), math.fabs(updates[1]-inits[1])
    inits = updates
    count += 1
    
print('EM converges in %s steps' %count)
print('Results:', inits)

EM converges in 5 steps
Results: [0.71253101 0.31411443]


In [ ]:
#Manh
import numpy as np
theta = [0.7,0.3]
heads = [14, 33, 19, 10, 0, 17, 24, 17, 1, 36, 5, 6, 5, 13, 4, 35, 5, 5, 74, 34]
throws = [41, 43, 23, 23, 1, 23, 36, 37, 2, 131, 5, 29, 13, 47, 10, 58, 15, 14, 100, 113]
def fac(k):
    if k == 0 or k == 1:
        return 1
    return k*fac(k-1)
def c(n,k):
    return fac(n)//(fac(k)*fac(n-k))


def EM(theta,heads,throws):
    rang = [0+0.01*i for i in range(1,100)]
    q = [[None for i in range(len(heads))] for i in range(2)]
    k = [None for i in range(len(heads))]
    old = theta
    real = [0,0]
    while (real[0]-old[0])**2+(real[1]-old[1])**2 > 0.001:
        for j in range(len(heads)):
            #Choosing the hidden variable by picking one that
            #is more likely with the current parameters
            q[0][j] = c(throws[j],heads[j])*theta[0]**heads[j]*(1-theta[0])**(throws[j]-heads[j])
            q[1][j] = c(throws[j],heads[j])*theta[1]**heads[j]*(1-theta[1])**(throws[j]-heads[j])
            
            if q[0][j] >= q[1][j]:
                k[j] = 0
            else:
                k[j] = 1
        throw,head = 0,0
        for cur in rang:
            x = 0
            for i in range(len(heads)):
                if k[i] == 0:
                    throw += throws[i]
                    head += heads[i]
            real[0] = head/throw
        throw,head = 0,0
        for cur in rang:
            x = 0
            for i in range(len(heads)):
                if k[i] == 1:
                    throw += throws[i]
                    head += heads[i]
            real[1] = head/throw
        old = real
    return real
print(EM(theta,heads,throws))
